In [2]:
import re
import requests
import urllib.request as img_request
import time
from tqdm import tqdm
import pandas as pd
import json 
import matplotlib.pyplot as plt
from PIL import Image
from os import makedirs
from os import listdir
from os import getcwd
import os

from sklearn.model_selection import train_test_split

# Init

Defining important directories as variables

In [3]:
cwd = getcwd()
data_dir = "data/"
img_dir = "data/images/"

Defining regex tags

In [ ]:
if not (os.path.exists(data_dir)):
    print("Making dir" , data_dir)
    makedirs(data_dir)

if not(os.path.exists(img_dir)):
    print("Making dir" , img_dir)
    makedirs(img_dir)

# Scraping

## Sailboatdata.com

In [ ]:
n_pages_tag = re.compile(r'<a href=.*page=(\d*).*/li>') # For getting the amount of pages on the domain

name_tag = re.compile(r'<a href=\"https://sailboatdata\.com/sailboat/.*\">(.*)</a>') # For making a list of all the boats on sailboatdata

specs_tag = re.compile(r'<div class=\" col-\w\w-\d*  col-\w\w-6 sailboatdata-label \">\s*(.*):\s</div>\s<.*\s*(.*)') # For scraping data from specific boat url

image_tag = re.compile(r'(?:photo|drawing)\".*src=\"(http.*)\"/>')

photo_draw_tag = re.compile(r'sailboat/(\w*)')

In [ ]:
specs = []
categories = []
boat_data = pd.DataFrame()
counter = 0

In [ ]:
general_url = 'https://sailboatdata.com/sailboat?page={}&paginate=25'

print("Connecting to" , general_url.format(1))
init_request = requests.get(general_url.format(1))
print(init_request.status_code)
pages = re.findall(n_pages_tag , init_request.text)
n_pages = pages[-1]

boat_types = []
for i in tqdm(range (1,int(n_pages)+1)):
  # print("Connecting to" , general_url.format(i))
  r = requests.get(general_url.format(i))
  # print(r.status_code)
  boat_types += re.findall(name_tag,r.text)
  time.sleep(0.5) #To not throw too many requests at the website

print(len(boat_types))

In [ ]:
for boat in boat_types:
    # GETTNG REQUEST
    print("Scraping",boat,"...")
    r = requests.get("https://sailboatdata.com/sailboat/{}?units=metric".format(boat.replace(" ","-").replace("(","").replace(")","").replace(".","")))
    raw_text = r.text

    # SCRAPING IMAGES
    images = re.findall(image_tag,raw_text)
    print("         -------scraping")
    for image in images:
        img_path = str(counter).zfill(5)+".jpg"
        if not os.path.isfile(img_path):
            counter += 1
            print(image)
            try:
                img_request.urlretrieve(image,img_dir+img_path)
            except:
                print("Strange url" , image.replace(" ","%20"))
                try:
                    img_request.urlretrieve(image,img_dir+img_path)
                except:
                    has_image = False

            # SCRAPING SPECS
            categories_specs = re.findall(specs_tag,raw_text)
            specs = [el[1] for el in categories_specs]
            categories = [el[0] for el in categories_specs]
            new_row = {categories[i]: specs[i] for i in range(len(categories))}
            new_row['name'] = boat
            new_row['img_path'] = img_path
            boat_data = pd.concat([boat_data, pd.DataFrame([new_row])], ignore_index=True)



    print("Done\n")
    if counter%100 == 1:
        print("saving")
        boat_data.to_csv(data_dir+"boat_data.csv" , index=False)
boat_data.to_csv(data_dir+"boat_data.csv" , index = False)

In [4]:
pd.set_option('display.max_columns', None)
data_raw = pd.read_csv(data_dir+"boat_data.csv")
data_raw

,Hull Type,Rigging Type,LOA,LWL,Beam,S.A. (reported),Draft (max),Displacement,Ballast,S.A./Disp.,Bal./Disp.,Disp./Len.,Construction,Ballast Type,First Built,# Built,Designer,Comfort Ratio,Capsize Screening Formula,S#,I,J,P,E,SPL/TPS,ISP,S.A. Fore,S.A. Main,S.A./Disp. (calc.),Est. Forestay Len.,Download Boat Record,name,img_path,Draft (min),Builder,Model,Last Built,Make,Type,HP,Fuel,Headroom,Water,Website,KSP,PY,EY,Bridgedeck Clearance,Mast Height from DWL,Bruce Number
0,Fin w/bulb & spade rudder,Fractional Sloop,10.30 m,8.20 m,2.50 m,41.81 m<sup>2</sup>,1.80 m,"1,600 kg",725 kg,31.16,45.31,80.89,"FG w/core,Composite",Lead,1990.0,350.0,Ron Holland & Rolf Gyhlenius,11.41,2.16,6.48,9.81 m,3.10 m,11.00 m,4.15 m,4.00 m,12.50 m,15.22 m<sup>2</sup>,22.83 m<sup>2</sup>,28.36,10.29 m,"<a class="""" href=""https://sailboatdata.com/sai...",11 METER,00000.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fin w/bulb & spade rudder,Fractional Sloop,10.30 m,8.20 m,2.50 m,41.81 m<sup>2</sup>,1.80 m,"1,600 kg",725 kg,31.16,45.31,80.89,"FG w/core,Composite",Lead,1990.0,350.0,Ron Holland & Rolf Gyhlenius,11.41,2.16,6.48,9.81 m,3.10 m,11.00 m,4.15 m,4.00 m,12.50 m,15.22 m<sup>2</sup>,22.83 m<sup>2</sup>,28.36,10.29 m,"<a class="""" href=""https://sailboatdata.com/sai...",11 METER,00001.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fin w/bulb & spade rudder,Fractional Sloop,7.32 m,5.49 m,1.27 m,14.59 m<sup>2</sup>,0.91 m,413 kg,136 kg,26.80,32.97,69.66,Plywood or GRP,Iron,1938.0,750.0,C. Raymond Hunt,10.58,1.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",110,00002.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fin w/bulb & spade rudder,Fractional Sloop,7.32 m,5.49 m,1.27 m,14.59 m<sup>2</sup>,0.91 m,413 kg,136 kg,26.80,32.97,69.66,Plywood or GRP,Iron,1938.0,750.0,C. Raymond Hunt,10.58,1.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",110,00003.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Centerboard Dinghy,Gunter,5.97 m,5.39 m,1.43 m,12.73 m<sup>2</sup>,0.29 m,231 kg,NaN,34.39,NaN,41.06,Wood,NaN,1931.0,NaN,Krogen Brothers,5.48,2.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",12 SQUARE METER SHARPIE,00004.jpg,0.97 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10490,Fin with rudder on skeg,Masthead Sloop,10.50 m,8.42 m,3.20 m,46.73 m<sup>2</sup>,1.80 m,"5,000 kg","2,200 kg",16.30,44.00,233.55,FG,NaN,1984.0,NaN,Sparkman & Stephens,25.06,1.89,2.26,13.41 m,3.95 m,11.80 m,3.43 m,NaN,NaN,26.49 m<sup>2</sup>,20.23 m<sup>2</sup>,16.30,13.98 m,"<a class="""" href=""https://sailboatdata.com/sai...",ZUANELLI 34,10490.jpg,NaN,Cantiere Zuanelli (ITA),NaN,NaN,Volvo Penta,Diesel,NaN,72 L,NaN,151 L,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10491,Fin w/spade rudder,Masthead Sloop,12.70 m,10.35 m,3.90 m,69.31 m<sup>2</sup>,1.95 m,"9,000 kg","3,600 kg",16.35,40.00,226.16,FG,Lead,1996.0,NaN,Sparkman & Stephens,28.35,1.89,2.34,16.15 m,4.88 m,14.50 m,4.12 m,NaN,NaN,39.40 m<sup>2</sup>,29.88 m<sup>2</sup>,16.34,16.87 m,"<a class="""" href=""https://sailboatdata.com/sai...",ZUANELLI 401,10491.jpg,NaN,Cantiere Zuanelli (ITA),NaN,NaN,Volvo Penta,Diesel,55,49 L,NaN,575 L,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10492,Fin Keel w/bulb,Fractional Sloop,5.79 m,5.31 m,1.88 m,17.00 m<sup>2</sup>,0.86 m,379 kg,80 kg,33.05,21.05,70.60,Wood/FG,Lead,1960.0,NaN,Ernst Lehfeld,6.40,2.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",ZUGVOGEL (KEEL),10492.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10493,Dbrd. Dinghy,Cat (unstayed),3.89 m,3.79 m,1.52

## boat24.com

regex

In [30]:
n_pages_tag = re.compile(r'\?page=(\d{3,})') # For getting the amount of pages on the domain

name_and_url_tag = re.compile(r'__title\"><a href=\"(https://www\.boat24\.com.+)\"\stitle=\"(.+?)\"\s>')

image_tag = re.compile(r'srcset=\"(https://[^\s]+?\.jpg) \d[^2]')


In [19]:
data_raw['img_path'][data_raw.__len__()-1]

'10494.jpg'

In [36]:
# general_url = 'https://www.boat24.com/en/secondhandboats/?src={}'

# counter = int(data_raw['img_path'][data_raw.__len__()-1].replace(".jpg",""))+1
# boat_data = pd.DataFrame()

# for name in data_raw['name'].values:
#     print("Scraping",name,"...")   
#     r = requests.get(general_url.format(name.replace(" ","+")))
#     names_and_url = re.findall(name_and_url_tag , r.text)
#     print(names_and_url)
#     if len(names_and_url) == 0:
#         pass
#     else:
#         for i in range(len(names_and_url)):
#             images = re.findall(image_tag,requests.get(names_and_url[i][0]).text)
#             for image in images:
#                 img_path = str(counter).zfill(5)+".jpg"
#                 if not os.path.isfile(img_path):
#                     counter += 1
#                     print(image)
#                     try:
#                         img_request.urlretrieve(image,img_dir+img_path)
#                     except:
#                         print("Strange url" , image.replace(" ","%20"))
#                         try:
#                             img_request.urlretrieve(image,img_dir+img_path)
#                         except:
#                             has_image = False
#                 new_row = {}
#                 new_row['name'] = name
#                 new_row['img_path'] = img_path
#                 boat_data = pd.concat([boat_data, pd.DataFrame([new_row])], ignore_index=True)

#     if counter%100 == 1:
#         print("saving")
#         boat_data.to_csv(data_dir+"boat_data_additional.csv" , index=False)
# boat_data.to_csv(data_dir+"boat_data_additional.csv" , index = False)


Scraping 11 METER ...
[]
Scraping 11 METER ...
[]
Scraping 110 ...
[('https://www.boat24.com/en/powerboats/performance/performance-1107-1201-refit-2022-carbon-edition/detail/508471/', 'Performance 1107 1201 REFIT 2022 Carbon Edition!'), ('https://www.boat24.com/en/powerboats/scand/scand-11000-dynamic/detail/529784/', 'Scand 11000 Dynamic'), ('https://www.boat24.com/en/powerboats/menorquin/menorquin-110/detail/520010/', 'Menorquin 110'), ('https://www.boat24.com/en/powerboats/performance/performance-1107/detail/532470/', 'Performance 1107'), ('https://www.boat24.com/en/powerboats/yachtwerft-barten-machetta-1100-l/detail/531837/', 'Yachtwerft-Barten Machetta 1100 L'), ('https://www.boat24.com/en/powerboats/ruber-110sl/detail/490619/', 'Ruber 110SL'), ('https://www.boat24.com/en/powerboats/lehmar/lehmar-lehmann-ch-ls1100/detail/464463/', 'Lehmar Lehmann CH, LS1100'), ('https://www.boat24.com/en/powerboats/menorquin/menorquin-110/detail/498874/', 'Menorquin 110'), ('https://www.boat24.com/

NameError: name 'names' is not defined

# Cleaning

In [4]:
units_cleaner_tag = re.compile(r'([\d.]+)(\s*m|\s*kg)(<.*>)*')

def clean_row(row):
  cleaned_row = []
  for i in range(len(row)):
    if i != 32:
      try:
        cleaned_row.append(float(re.sub(units_cleaner_tag , r'\g<1>' , row[i].replace(',','') , )))
      except:
        cleaned_row.append(row[i])
    else:
      cleaned_row.append(row[i])
  return cleaned_row

In [5]:
cleaned_data = []
for i , row in data_raw.iterrows():
  cleaned_data.append(clean_row(row))

In [6]:
data_clean = pd.DataFrame(cleaned_data , columns = data_raw.columns)
data_clean.to_csv(data_dir+"boat_data_clean.csv" , index = False)

In [12]:
data_clean = pd.read_csv(data_dir+"boat_data_clean.csv")

In [7]:
row_clean_tag = re.compile(r'[^\w\s/&]')
for column in data_clean.columns:
    if (data_clean[column].dtype == "object" or data_clean[column].dtype == "category") and not column == "Download Boat Record" and not column == "url" and not column == "img_path":
        data_clean[column] = data_clean[column].str.strip().str.rstrip('.').str.lower()
        data_clean[column] = data_clean[column].str.replace(row_clean_tag , '')
        # data_clean[column] = data_clean[column].astype('category')
        # pd.DataFrame(data_clean[column].cat.categories).to_csv(data_dir+"labels/"+column.replace(" ","_")+".txt" , index = False , header = False)
        # data_clean[column] = data_clean[column].cat.codes

# Splitting

In [8]:
data_train , data_test = train_test_split(data_clean , test_size = 0.2 , random_state = 42)


In [ ]:
train_dir = "data/train/"
test_dir = "data/test/"

if not(os.path.exists(train_dir)):
    print("Making dir" , train_dir)
    makedirs(train_dir)

if not(os.path.exists(test_dir)):
    print("Making dir" , test_dir)
    makedirs(test_dir)


for image in data_train['img_path']:
    try:
        img = Image.open(img_dir+image)
        img = img.convert('RGB')
        img.save(train_dir+image)
    except:
        print("Image not found" , image)
        data_train = data_train[data_train['img_path'] != image]

for image in data_test['img_path']:
    try:
        img = Image.open(img_dir+image)
        img = img.convert('RGB')
        img.save(test_dir+image)
    except:
        print("Image not found" , image)
        data_test = data_test[data_test['img_path'] != image]


In [9]:
data_train.to_csv(data_dir+"boat_data_train.csv" , index = False)
data_test.to_csv(data_dir+"boat_data_test.csv" , index = False)

# Plotting

In [ ]:
# data_train = pd.read_csv(data_dir+"boat_data_train.csv")
# data_test = pd.read_csv(data_dir+"boat_data_test.csv")
# data_clean = pd.read_csv(data_dir+"boat_data_clean.csv")

In [ ]:
plot_dir = "plots/"
if not(os.path.exists(plot_dir)):
    print("Making dir" , plot_dir)
    os.makedirs(plot_dir)

In [ ]:
train_test = ["train" , "test"]
size_factor = [1,0.2]
for i , data in enumerate([data_train , data_test]):
    for column in data_clean.columns:
        if not column == "Download Boat Record" and not column == "name" and not column == "url" and not column == "img_path":
            fig = plt.figure(figsize=(10*2,5*2))
            if data[column].dtype == "object":
                to_remove = 50*size_factor[i]
                try:
                    dat = data[column].value_counts()
                    # dat = dat[dat > to_remove] # Removing entries with less than to_remove entries
                    dat = dat/len(data[column]) # Normalising
                    print("Plotting" , column)
                    dat.to_csv(plot_dir+column.replace('/',"_")+"_"+train_test[i]+".csv")
                    dat.plot(kind='barh')
                    plt.annotate(text = "Total entries were "+str(len(data[column].value_counts())) , xy = (0.8,0.95) , xycoords = "axes fraction")
                    plt.annotate(text = "Removed entries where n<"+str(to_remove) , xy = (0.8,0.9) , xycoords = "axes fraction")
                except:
                    data[column].value_counts().plot(kind='barh')
                fig.name = column
                fig.suptitle(column+str(" ")+train_test[i] , fontsize=16)
                path = plot_dir+column.replace('/',"_")+"_"+train_test[i]+".png"
                plt.savefig(path)
                fig.clear()


            # else:
                # data[column].plot(kind='hist' , logy=True)
            # fig.name = column
            # fig.suptitle(column+str(" ")+train_test[i] , fontsize=16)
            # path = plot_dir+column.replace('/',"_")+"_"+train_test[i]+".png"
            # plt.savefig(path)
            # fig.clear()


Plotting Hull Type
Plotting Rigging Type
Plotting Construction
Plotting Ballast Type
Plotting Designer


C:\Users\chris\AppData\Local\Temp\ipykernel_5516\2040015531.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10*2,5*2))


Plotting Builder
Plotting Model
Plotting Make
Plotting Type
Plotting HP
Plotting Fuel
Plotting Water
Plotting Website
Plotting Hull Type
Plotting Rigging Type
Plotting Construction
Plotting Ballast Type
Plotting Designer
Plotting Builder
Plotting Model
Plotting Make
Plotting Type
Plotting HP
Plotting Fuel
Plotting Water
Plotting Website


<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

...

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

# Label embedding

In [31]:
data_clean = pd.DataFrame(cleaned_data , columns = data_raw.columns)

col_list = ['Hull Type' , 'Rigging Type' , 'Construction' , 'Ballast Type' , 'Designer' , 'name']

for column in data_clean.columns:
    if column in col_list:
        path = data_dir+"/labels/"+column.replace(" ","_")
        data_clean[column] = data_clean[column].astype('category')
        classes = dict( zip(data_clean[column] , data_clean[column].cat.codes))
        with open(path+".json", "w") as outfile:
            json.dump(classes, outfile , indent = 4)
        data_clean[column] = data_clean[column].cat.codes

data_clean.to_csv(data_dir+"boat_data_clean.csv" , index = False)
data_train , data_test = train_test_split(data_clean , test_size = 0.2 , random_state = 42)
data_train.to_csv(data_dir+"boat_data_train.csv" , index = False)
data_test.to_csv(data_dir+"boat_data_test.csv" , index = False)

In [23]:
# file_path = "images/00002.jpg"
# df = data_train
# idx = df.index[(df['img_path'] == file_path.split("/")[1])].values[0]

# print(
# Hull_Type_Classes[df['Hull Type'][idx]],[df['Hull Type'][idx]],"\n"+
# Rigging_Type_Classes[df['Rigging Type'][idx]],[df['Rigging Type'][idx]],"\n"+
# Construction_Classes[df['Construction'][idx]],[df['Construction'][idx]],"\n"+
# Ballast_Type_Classes[df['Ballast Type'][idx]],[df['Ballast Type'][idx]],"\n"+
# Designer_Classes[df['Designer'][idx]],[df['Designer'][idx]],"\n"+
# Name_Classes[df['name'][idx]],[df['name'][idx]],"\n"
# )

fin wbulb  spade rudder [19] 
fractional 78 sloop [14] 
plywood or grp wfoam sandwich [421] 
internal cast iron centerboard galvanized steel [37] 
c raymond hunt   assoc [298] 
11 meter [0] 

